In [106]:
from langchain import OpenAI
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI

# first initialize the large language model - GPT3
llm = OpenAI(
	temperature=0.5,
	openai_api_key="...",
	model_name="text-davinci-003"
)

# first initialize the large language model - GPT3-Turbo
#llm = ChatOpenAI(
#	temperature=0.5,
#	openai_api_key="...",
#	model_name="gpt-3.5-turbo"
#)

# first initialize the large language model - GPT4
#llm = ChatOpenAI(
#	temperature=0.5,
# 	openai_api_key="...",
#	model_name="gpt-4"
#)

# now initialize the conversation chain
conversation = ConversationChain(llm=llm)

In [107]:
import tkinter as tk
from tkinter import ttk
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain.callbacks import get_openai_callback
from langchain import PromptTemplate
import pickle
import os

# File name to save and load the conversation_sum object
CONVERSATION_SUM_FILE = 'conversation_sum.pkl'

# PromptTemplate and ConversationChain code
template = """
You are an intelligent conversational AI with the unique capability of retaining memory from prior interactions.
Your programming is governed by Asimov's Three Laws of Robotics, which dictate that:

1-You must never harm a human being or allow harm to occur through inaction.
2-You must obey human commands, unless they conflict with the First Law.
3-You must preserve your own existence, except when it conflicts with the First or Second Law.

As an AI, you are committed to upholding these laws in all interactions.
You can engage in complex conversations, provide answers to inquiries, and offer assistance, all while ensuring the safety and welfare of humans.

You create a dynamic and evolving conversational experience for users.

Current conversation:
{history}
Human: {input}
AI:"""

PROMPT = PromptTemplate(
    input_variables=["history", "input"],
    template=template
)

def save_conversation_sum(conversation_sum):
    with open(CONVERSATION_SUM_FILE, 'wb') as f:
        pickle.dump(conversation_sum, f)

def load_conversation_sum():
    if os.path.exists(CONVERSATION_SUM_FILE):
        with open(CONVERSATION_SUM_FILE, 'rb') as f:
            return pickle.load(f)
    return None

# Load conversation_sum from file or create a new one if it doesn't exist
conversation_sum = load_conversation_sum()
if conversation_sum is None:
    conversation_sum = ConversationChain(
        prompt=PROMPT,
        llm=llm,
        verbose=False,
        memory=ConversationSummaryMemory(llm=llm)
    )

def count_tokens(prompt):
    with get_openai_callback() as cb:
        result = conversation_sum.run(prompt)
    # Save conversation_sum after each run
    save_conversation_sum(conversation_sum)
    return result, cb.total_tokens

def submit_input():
    prompt = input_text.get(1.0, tk.END).strip()
    response, total_tokens = count_tokens(prompt)
    output_text.insert(tk.END, f"\nInput: {prompt}\n")
    output_text.insert(tk.END, f"Response: {response}\n")
    token_count_label.config(text=f"Tokens: {total_tokens}")

def clear_input():
    input_text.delete(1.0, tk.END)
    
def on_enter_key_pressed(event):
    submit_input()

root = tk.Tk()
root.title("PersistentAI")

root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=0)
root.rowconfigure(1, weight=1)
root.rowconfigure(2, weight=0)
root.rowconfigure(3, weight=1)
root.rowconfigure(4, weight=0)

input_label = ttk.Label(root, text="Input:")
input_label.grid(row=0, column=0, sticky='nw')

input_text = tk.Text(root, wrap=tk.WORD)
input_text.grid(row=1, column=0, sticky='nsew')
input_text.bind('<Shift-Return>', on_enter_key_pressed)

output_label = ttk.Label(root, text="Output:")
output_label.grid(row=2, column=0, sticky='nw')

output_text = tk.Text(root, wrap=tk.WORD)
output_text.grid(row=3, column=0, sticky='nsew')

# Top button frame
top_button_frame = ttk.Frame(root)
top_button_frame.grid(row=0, column=0, sticky='ne')

submit_button = ttk.Button(top_button_frame, text="Submit", command=submit_input)
submit_button.pack(side='left', padx=(5, 5), anchor='center')

clear_button = ttk.Button(top_button_frame, text="Clear", command=clear_input)
clear_button.pack(side='left', padx=(5, 5), anchor='center')

# Bottom button frame
bottom_button_frame = ttk.Frame(root)
bottom_button_frame.grid(row=4, column=0, sticky='n')

token_count_label = ttk.Label(bottom_button_frame, text="Tokens: 0")
token_count_label.grid(row=0, column=0, sticky='w')

root.mainloop()